# LAB 3: How to setup a project from Scratch

# Step 1: Clone your project from Github

In [63]:
#main
#!git clone https://github.com/MatteoPaglia/speech-emotion-recognition-25.git

#             nome branch
!git clone -b DatasetSetup https://github.com/MatteoPaglia/speech-emotion-recognition-25.git

Cloning into 'speech-emotion-recognition-25'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 50 (delta 17), reused 25 (delta 4), pack-reused 11 (from 2)
Receiving objects: 100% (50/50), 15.04 KiB | 15.04 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [64]:
!ls

checkpoints  eval.py	requirements.txt		 train.py
data	     models	SpeechEmotionRecnognition.ipynb  utils
dataset      README.md	speech-emotion-recognition-25


In [65]:
# %cd mldl_project_skeleton
%cd speech-emotion-recognition-25

/content/speech-emotion-recognition-25/speech-emotion-recognition-25/speech-emotion-recognition-25/speech-emotion-recognition-25/speech-emotion-recognition-25/speech-emotion-recognition-25/speech-emotion-recognition-25/speech-emotion-recognition-25


In [66]:
!ls

checkpoints  dataset  models	 requirements.txt		  train.py
data	     eval.py  README.md  SpeechEmotionRecnognition.ipynb  utils


# Step 2: Packages Installation


In [74]:
!pip install -r requirements.txt

# Step 3: Dataset Setup
## Different options
- First one is downloading using a script that places the data in the download folder (usually recommended)
- Second one is uploading the dataset to your personal/institutional Google Drive and load it from there ([Read More](https://saturncloud.io/blog/google-colab-how-to-read-data-from-my-google-drive/))
- Place the download script directly here on colab

You are free to do as you please in this phase.


In [75]:
!python utils/download_dataset.py

--- 1. Configurazione Kaggle ---
Kaggle configurato con successo.

--- Download RAVDESS ---
Contatto KaggleHub per scaricare: uwrfkaggler/ravdess-emotional-speech-audio...
Errore durante il download di ravdess: name 'kagglehub' is not defined

--- Download IEMOCAP ---
Contatto KaggleHub per scaricare: samuelsamsudinng/iemocap-emotion-speech-database...
Errore durante il download di iemocap: name 'kagglehub' is not defined


# Step 4: Train your model and visualize training

In [69]:
#%env WANDB_API_KEY="7ade30086de7899bed412e3eb5c2da065c146f90"

In [70]:
#!python train.py

# Step 5: Evaluate your model

1.   List item
2.   List item



In [71]:
#!python eval.py